In [1]:
import matplotlib
matplotlib.use("nbagg")

import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

from utility import *
from model import * 

import torch
import torch.nn as nn
import torch.utils as utils
import torch.nn.init as init
import torch.utils.data as data
import torchvision.utils as v_utils
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.autograd import Variable

# Import Data 

In [2]:
from os import listdir
from os.path import isfile, join
import numpy as np

polycube_path = "/Users/davidcleres/DeepShape/Polycubing-Automated/Generated-Cars-Grid_size64/"
polycube_files = [f for f in listdir(polycube_path) if isfile(join(polycube_path, f))]

voxelized_mesh_path = "/Users/davidcleres/DeepShape/Polycubing-Automated/voxelizedMeshes-Grid_size64/"
voxelized_mesh_files = [f for f in listdir(voxelized_mesh_path) if isfile(join(voxelized_mesh_path, f))]

voxelizedFiles = []
polycubedFiles = []

for f in voxelized_mesh_files: 
    if f[-13:] == "voxelized.txt":
        voxelizedFiles = np.hstack((voxelizedFiles, f))
    
for f in polycube_files:
    if f[-14:] == "finalCubes.txt":
        polycubedFiles = np.hstack((polycubedFiles, f))

In [3]:
grid_size = 64
voxelized_train_input, polycube_target=loadData(grid_size, polycube_path, voxelized_mesh_path, voxelizedFiles, polycubedFiles, loadFromScratch=True)

<class 'torch.IntTensor'> torch.Size([64, 64, 64])
<class 'torch.IntTensor'> torch.Size([64, 64, 64])
<class 'torch.IntTensor'> torch.Size([64, 64, 64])
<class 'torch.IntTensor'> torch.Size([64, 64, 64])
<class 'torch.IntTensor'> torch.Size([64, 64, 64])
<class 'torch.IntTensor'> torch.Size([64, 64, 64])
<class 'torch.IntTensor'> torch.Size([64, 64, 64])
<class 'torch.IntTensor'> torch.Size([64, 64, 64])
<class 'torch.IntTensor'> torch.Size([64, 64, 64])
<class 'torch.IntTensor'> torch.Size([64, 64, 64])
<class 'torch.IntTensor'> torch.Size([64, 64, 64])
<class 'torch.IntTensor'> torch.Size([64, 64, 64])
<class 'torch.IntTensor'> torch.Size([64, 64, 64])
<class 'torch.IntTensor'> torch.Size([64, 64, 64])
<class 'torch.IntTensor'> torch.Size([64, 64, 64])
<class 'torch.IntTensor'> torch.Size([64, 64, 64])
<class 'torch.IntTensor'> torch.Size([64, 64, 64])
<class 'torch.IntTensor'> torch.Size([64, 64, 64])
<class 'torch.IntTensor'> torch.Size([64, 64, 64])
<class 'torch.IntTensor'> torch

In [4]:
import math #Really need thi import ????? 

batch_size = 15 
preprocessed_input_train, preprocessed_input_validation, preprocessed_input_train_target, preprocessed_input_validation_target = preprocessing_train(voxelized_train_input, polycube_target,batch_size, False, False)

preprocessed_input_train = torch.from_numpy(preprocessed_input_train)
preprocessed_input_validation = torch.from_numpy(preprocessed_input_validation)
preprocessed_input_train_target = torch.from_numpy(preprocessed_input_train_target)
preprocessed_input_validation_target = torch.from_numpy(preprocessed_input_validation_target)

Ntrain = len(preprocessed_input_train[:, 0,0,0,0]) 
Nvalidation = len(preprocessed_input_validation[:,0,0,0,0])
image_size = 64

train_input = Variable(preprocessed_input_train.view(Ntrain, 1, image_size, image_size, image_size).float())
validation_input = Variable(preprocessed_input_validation.view(Nvalidation, 1, image_size, image_size, image_size).float())

labels_train = Variable(preprocessed_input_train_target.view(Ntrain, 1, image_size, image_size, image_size), requires_grad=False).float() 
labels_validation = Variable(preprocessed_input_validation_target.view(Nvalidation, 1, image_size, image_size, image_size), requires_grad=False).float() 

print('train', train_input.shape)
print('validation', validation_input.shape)
print('train_target', labels_train.shape)
print('validation_target', labels_validation.shape)

train torch.Size([45, 1, 64, 64, 64])
validation torch.Size([15, 1, 64, 64, 64])
train_target torch.Size([45, 1, 64, 64, 64])
validation_target torch.Size([15, 1, 64, 64, 64])


In [5]:
model = nn.Conv3d(in_channels=1, out_channels=1, kernel_size=3, stride=1, padding=1,bias=True)
model(train_input).size()

torch.Size([45, 1, 64, 64, 64])

In [6]:
unet = UnetGenerator_3d(in_dim=1,out_dim=1,num_filter=4)

#output = unet(train_input)
#print(output.size()) 


------Initiating U-Net------



In [7]:
import torch.optim as optim
import torch.nn.functional as F

Nepochs = 5
Nrep = 1

# Train network 
criterion = nn.BCELoss()
#criterion = nn.CrossEntropyLoss()
#criterion = nn.PoissonNLLLoss()
#criterion = nn.BCEWithLogitsLoss()
#criterion = nn.SmoothL1Loss() #interesting ... but does not converge
#criterion = nn.MSELoss() #0.83 but unstable

optimizer = optim.SGD(model.parameters(), lr=1e-2, momentum=0.90)
#optimizer = optim.Adam(model.parameters())
#optimizer = optim.Adagrad(model.parameters())
#optimizer = optim.Adamax(model.parameters())
#optimizer = optim.ASGD(model.parameters())
#optimizer = optim.RMSprop(model.parameters())
#optimizer = optim.Rprop(model.parameters())

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=10, verbose=True)

train_errors = torch.Tensor(Nepochs).zero_()
validation_errors = torch.Tensor(Nepochs).zero_()

ep_loss = torch.Tensor(Nepochs).zero_()

for i_rep in range(Nrep):
    for i_ep in range(Nepochs):
        for b_start in range(0, Ntrain, batch_size):
            bsize_eff = batch_size - max(0, b_start+batch_size-Ntrain)  # boundary case
            unet.train()
            unet.zero_grad()
            output = unet(train_input.narrow(0, b_start, bsize_eff))
            print('output', output.shape)
            print('labels', labels_train.narrow(0, b_start, bsize_eff).shape)
            batch_loss = criterion(output, labels_train.narrow(0, b_start, bsize_eff))            
            ep_loss[i_ep] += batch_loss.data[0]
            batch_loss.backward()
            optimizer.step()
        
        scheduler.step(ep_loss[i_ep])
        
        '''
        nb_train_errs = compute_nb_errors(unet, train_input, labels_train, batch_size)
        nb_validation_errs = compute_nb_errors(unet, validation_input, labels_validation, batch_size)
        
        print("train_error", nb_train_errs)
        print("nb_validation_errs", nb_validation_errs)
        
        print("Epoch Number : ", i_ep)
        print("\t Training accuracy: ", np.abs(100*(Ntrain*100*100*100-int(nb_train_errs))/(Ntrain*100*100*100)))
        print("\t Validation accuracy ", np.abs(100*(Nvalidation*100*100*100-int(nb_validation_errs))/(Nvalidation*100*100*100))) 
        
        print("\t Epoch Loss ", ep_loss[i_ep])
        
        train_errors[i_ep] = int(nb_train_errs)
        validation_errors[i_ep] = int(nb_validation_errs)
        '''

output torch.Size([15, 1, 64, 64, 64])
labels torch.Size([15, 1, 64, 64, 64])


KeyboardInterrupt: 